<div align="center">
    <h1><strong>Transferencia de Calor 1D</strong></h1>
    <h2>Juan David Argüello Plata</h2>
    <img src="Images/uis.png" style="width: 300px;" />
</div>

## Planteamiento del problema

<div align="justify">
    Se busca analizar, mediante métodos <u>analíticos</u> y <u>numéricos</u>, la transferencia de calor en una barra cuadrangular bajo las siguientes suposiciones:
</div>

* Conductividad térmica constante.
* Conducción 1D (dirección x - línea verde en la Figura 1).
* Conducción estable (no transitoria).
* Generación de calor volumétrica constante.

<img src="Images/Esquema_3D/Esquema.PNG" style="width: 700px;" />
<center><i>Figura 2.</i> Transferencia de calor por conducción.</center>

## __1. Desarrollo teórico__

### 1.1. Sobre toda la barra

<div align="justify">
    Del balance energético sobre toda la barra se obtiene lo siguiente:
    $$
    \begin{equation}
        Q _g A x + K A \frac{dT}{dx} = 0
        \tag{1}
        \label{Global}
    \end{equation}
    $$
</div>

In [ ]:
from sympy import init_session
init_session(use_latex=True)

### Condiciones de frontera

<div align="justify">
    Las condiciones de frontera sobre el dominio $\left(0 \leq x \leq L\right)$ son:
    <ul>
        <li>$T (0) = T_0 \rightarrow$ Temperatura inicial.</li>
        <li>$q(L) = q_L = -K \frac{\partial T}{\partial x} \Big| _{x=L} = 0 \rightarrow$ Transferencia de calor por unidad de área en el extremo.</li>
    </ul>
</div>

In [ ]:
"""
    Selecciona el valor de la temperatura inicial, 
    la generación de calor, la conductividad térmica 
    y la longitud.
"""
var = {
    'T_0':{
        'Valor':0,
        'Units': '°C'
    },
    'Q_g':{
        'Valor':100,
        'Units': 'W/m^3'
    },
    'K':{
        'Valor': 55,
        'Units': 'W/°C'
    },
    'L':{
        'Valor':3,
        'Units':'m'
    }
}

from IPython.display import HTML, display
Prim = True
for const in var:
    if Prim:
        text = const[0] + '(0) = T_0 = '
        Prim = False
    else:
        text = const + '= '
    display(HTML('$' + text + str(var[const]['Valor']) + \
                 '\left[' + var[const]['Units'] + '\\right]$'))

<div align="justify">
    Resolviendo la Ecuación 1 mediante <i>Sympy</i> (librería de Python), y empleando las condiciones de frontera respectivas, se obtiene la siguiente relación matemática para la temperatura en cualquier punto $x$.
</div>

In [ ]:
T = Function('T')
Q_g, A, K = symbols('Q_g A K')
Ec = Eq(Q_g*A*x + K*A*T(x).diff(x),0)
sol = dsolve(Ec,ics={T(0):var['T_0']['Valor']})
sol

<div align="justify">
    Sustituyendo los valores de generación de calor y conductividad térmica...
</div>

In [ ]:
constants = {Q_g:var['Q_g']['Valor'], K:var['K']['Valor']}
final = sol.subs(constants)
final

<div align="justify">
    Que gráficamente representa lo siguiente:
</div>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
sol = solve(final, T(x))
plot(sol[0], (x,0,var['L']['Valor']))

### 1.2. Sobre un Elemento Infinitesimal

<div align="justify">
    Desarrollando el balance de energía sobre el volumen de control inifinitesimal, se obtiene:
    $$
    \begin{equation}
        0 = \frac{d q_x}{d x} \Delta x - Q \Delta x \Delta y \Delta z
        \tag{2}
        \label{General}
    \end{equation}
    $$
    Dónde: $\Delta x$, $\Delta y$ y $\Delta z$ son las longitudes inifinitesimales; y $Q$ es calor generado, por unidad de volumen, del elemento.
</div>

<div align="justify">
    La Ecuación \ref{General} se puede relacionar con la Ley de Fourier (Ecuación \ref{fou}).
    $$
    \begin{equation}
        q_x = -K \frac{d T}{d x} \Delta y \Delta z
        \tag{3}
        \label{fou}
    \end{equation}
    $$
</div>

<div align="justify">
    Obteniéndose:
    $$
    \begin{equation}
        \left[ -K \frac{d ^2 T}{d x ^2} - Q \right] \Delta x \Delta y \Delta z = 0 \rightarrow K \frac{d ^2 T}{d x ^2} + Q = 0
        \tag{4}
        \label{DifGen}
    \end{equation}
    $$
    La Ecuación \ref{DifGen} corresponde a la conservación de la energía dentro del volumen de control infinitesimal. Resolviendo mediante <i>Sympy</i>, se obtiene lo siguiente:
</div>

In [ ]:
Ec = Eq(K*T(x).diff(x,2) + Q_g,0)
sol = dsolve(Ec, ics={T(0):var['T_0']['Valor'], T(x).diff(x).subs(x,var['L']['Valor']):0})
sol

In [ ]:
constants = {Q_g:var['Q_g']['Valor'], K:var['K']['Valor']}
final = sol.subs(constants)
final

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
sol = solve(final, T(x))
plot(sol[0], (x,0,var['L']['Valor']))

## __2. Solución Numérica__

<div align="justify">
    El proceso de solución se basa en la <i>discretización</i>, que consiste en reducir el problema a la determinación de la <u>temperatura</u> en <u>puntos específicos</u> (<strong>"nodos"</strong>).
</div>

<br>

<img src="Images/Esquema/Esquema.PNG" style="width: 700px;" />
<center><i>Figura 3.</i> Discretización del problema.</center>

<div align="justify">
    De los nodos se puede definir la función de temperatura $T(x)$, donde los valores de temperatura entre nodos se obtienen mediante interpolación. Al final, se esperaría obtener una gráfica de temperatura como la mostrada en la Figura 4.
</div>

<img src="Images/Gráficas/Temp.PNG" style="width: 700px;" />
<center><i>Figura 4.</i> Gráfica de temperatura final esperada.</center>

<div align="justify">
    Este problema en particular tiene <strong>4 nodos y 3 elementos</strong>. Como se puede observar en la Figura 3, $T(x)$ es una función <i>a trozos</i>.
</div>

### 2.1. FDM

<div align="justify">
    El desarrollo mediante el <i>Método de Diferencias Finitas</i> consiste en la conversión de la Ecuación 4 en un sistema matricial nodal, como se puede apreciar a continuación.
</div>

In [ ]:
#Número de nodos
n = 4
Ts = symbols('T_0:' + str(n))
l = Symbol('\Delta x')
#Condiciones internas
Ec = []
for i in range(1,n-1):
    Ec.append(Eq(K*(Ts[i] - 2*Ts[i-1] + Ts[i+1])/l**2 + Q_g,0))
#Condición de frontera
Ec.append(Eq(Ts[0], var['T_0']['Valor']))
Ec.append(Eq(-K*(Ts[-1] - Ts[-2])/l,0))
Ec

<div align="justify">
    La solución del sistema de ecuaciones mostrado es el siguiente:
</div>

In [ ]:
Ec = [Ts[0] - var['T_0']['Valor'], -K*(Ts[-1] - Ts[-2])/(var['L']['Valor'])]
for i in range(1,n-1):
    Ec.append(var['K']['Valor']*(Ts[i] -2*Ts[i-1] + Ts[i+1])/((var['L']['Valor'])/(n-1))**2 +\
              var['Q_g']['Valor'])
sol = linsolve(Ec, Ts)
sol

### 2.2. FEM

<div align="justify">
    La lógica empleada en la solución numérica es la siguiente:
</div>

<img src="Images/Esquema/Logic-1.png" style="width: 800px;" />
<center><i>Figura 5.</i> Lógica de solución.</center>

<div align="justify">
    En términos matemáticos, se resume en la solución de la ecuación matricial:
    $$
    \begin{equation}
        \left[ K \right] \lbrace{ T \rbrace} = \lbrace{ Q \rbrace}
        \tag{5}
        \label{EcGeneral}
    \end{equation}
    $$
    Dónde $K$ corresponde a la matriz de rigidez, $T$ al vector de temperatura y $Q$ al vector calor.
</div>

### ¿Cómo derivar el sistema de ecuaciones algebraicas?

<div align="justify">
    De la sección <i>Desarrollo teórico</i>, la Ecuación \ref{DifGen} muestra la forma diferencial del problema. La <strong>forma integral ponderada</strong> se puede apreciar en la Ecuación \ref{IntPond}.
    $$
    \begin{equation}
        \int _0 ^L w \left(K \frac{\partial ^2 T}{\partial x ^2} + Q \right) \partial x = 0
        \tag{6}
        \label{IntPond}
    \end{equation}
    $$
    Dónde $w(x)$ es una función arbitraria. Cabe destacar que <i>la solución exacta se cumple para cualquier función arbitraria</i>.
</div>

<br>

<div align="justify">
    Una forma "peculiar" de la Ecuación \ref{IntPond} es:
    $$
    \begin{equation}
        \int _0 ^L w^e \left(K \frac{\partial ^2 T}{\partial x ^2} + Q \right) \partial x = 0
        \tag{7}
        \label{IntPec}
    \end{equation}
    $$
    Dónde $w^e (x)$ es una función polinómica a trozos.
</div>

<br>

<div align="justify">
    Entre más nodos se usen, $w^e$ tenderá a ser $w$. La clave está en encontrar una función arbitraria que satisfaga la <i>función de forma</i>.
</div>

<img src="Images/Gráficas/w.PNG" style="width: 800px;" />
<center><i>Figura 6.</i> Función arbitraria.</center>

### ¿Cómo encontrar las temperaturas nodales?

<div align="justify">
    Cada ecuación relaciona la temperatura nodal con sus vecinos (ver Figura 4). Para solucionar la Ecuación \ref{IntPec}, se desarrolla <a href ="https://es.wikipedia.org/wiki/M%C3%A9todos_de_integraci%C3%B3n#M%C3%A9todo_de_integraci%C3%B3n_por_partes"><i>integración por partes</i></a> $\left( \int u dv = uv - \int v du \right)$.
    $$
    \begin{equation}
        w^e K \frac{\partial T}{\partial x} \Big| _0 ^L  - K \int _0 ^L \frac{\partial w^e}{\partial x} \frac{\partial T}{\partial x} \partial x + \int _0 ^L w^e Q \partial x = 0
        \tag{8}
        \label{EcNum}
    \end{equation}
    $$
    La Ecuación \ref{EcNum} se conoce como la <i>forma débil</i> de la ecuación diferencial original.
</div>

<br>

<div align="justify">
    A continuación, se desarrolla un análisis por términos de la Ecuación \ref{EcNum}.
    <ol>
        <li> El primer término, permite observar si se cumple la ley de la conservación de la energía.
            $$
                w^e K \frac{\partial T}{\partial x} \Big| _0 ^L = K \left[ w_4 \left( \frac{\partial T}{\partial x} \right) _4 - w_1 \left( \frac{\partial T}{\partial x} \right) _1 \right]
            $$
        </li>
        <li> El segundo término compete al desarrollo de la matriz de rigidez (Ecuación \ref{EcGeneral}).
        </li>
        <li>El tercer término corresponde al <i>"término fuente"</i>. La integración se realiza <u>elemento por elemento</u> (ver Figuras 6 y 7).
            $$
                \int _0 ^L w^e Q \partial x = Q \sum _{i=0} ^{n} \int _{i \Delta x} ^{\left(i+1 \right) \Delta x} w^e \partial x
            $$
            Dónde $n=\# Elementos$. Por ejemplo: para el primer elemento...
            $$
              Q \int _0 ^{\Delta x} w^e \partial x = Q \frac{\left(w_1 + w_2 \right) \Delta x}{2}
            $$
            <img src="Images/Esquema/part.PNG" style="width: 800px;" />
            
<center><i>Figura 7.</i> Distribución de energía por elemento.</center>
        </li>
    </ol>
</div>